# Mysql 连接对象

In [1]:
import pymysql


class DBHelper:
    def __init__(self):
        # 链接数据库
        try:
            # charset 默认是 latin1, 查询到中文会是？？
            # charset='utf8mb4' 避免有表情时插入错误
            self.__db = pymysql.connect(
                host='127.0.0.1',
                user='root',
                password='111111',
                database='test',
                charset='utf8mb4')
            self.__cur = self.__db.cursor()
        except pymysql.Error as e:
            print('链接数据库失败：', e.args[0], e.args[1])

    def insert(self, table, myDict):
        # 答案中存在表情会出错
        # 答案中存在双引号会出错，sql语句会发生歧义
        # 插入一条数据
        try:
            cols = ','.join(myDict.keys())
            values = ','.join(
                map(lambda x: '"' + str(x) + '"', myDict.values()))
            sql = 'INSERT INTO %s (%s) VALUES (%s)' % (table, cols, values)
            result = self.__cur.execute(sql)
            self.__db.commit()
        except pymysql.Error as e:
            print('插入失败：', e.args[0], e.args[1])
            # 发生错误时回滚
            # DML 语句，执行完之后，处理的数据，都会放在回滚段中（除了 SELECT 语句），
            # 等待用户进行提交（COMMIT）或者回滚 （ROLLBACK），当用户执行 COMMIT / ROLLBACK后，
            # 放在回滚段中的数据就会被删除。
            self.__db.rollback()

    def query(self, sql):
        try:
            self.__cur.execute(sql)
            result = self.__cur.fetchall()
            self.__db.commit()
            if result:
                return result
            else:
                return None
            
        except  pymysql.Error as e:
            print("数据库-查询异常")
            
    def check_exist(self, table, record):
        try:
            sql = 'SELECT COUNT(*) FROM %s WHERE url= %s' % (table, record)
            result = self.__cur.execute(sql)
            if result > 0:
                return True
            else:
                return False
        except pymysql.Error as e:
            print('查询失败！')
            return False
            
            
    def close(self ):
        self.__cur.close()
        self.__db.close()


# 爬取每个tag的首页地址

In [37]:
import requests
from bs4 import BeautifulSoup

def getAllTagsFirstLink():
    tag_list = []
    html = requests.get('http://www.meizitu.com')
    html.encoding = 'gbk'
    bsoup = BeautifulSoup(html.text.strip().replace('<!--   ','').replace('-->',''), "html.parser")
    tags = bsoup.find('div',{'class': 'tags'}).findAll('a')
    dbhelper = DBHelper()
    for a in tags:
        myDict = {
            'title': a.attrs['title'],
            'url':a.attrs['href']
        }
        tag_list.append(myDict)
        #print(a.attrs['title'],a.attrs['href'])
        dbhelper.insert('t_meizitu_tags_url',myDict)
    dbhelper.close()
    return tag_list
getAllTagsFirstLink()

[{'title': '颜值控', 'url': 'http://www.meizitu.com/a/pure.html'},
 {'title': '萌妹', 'url': 'http://www.meizitu.com/a/cute.html'},
 {'title': '性感', 'url': 'http://www.meizitu.com/a/sexy.html'},
 {'title': '福利', 'url': 'http://www.meizitu.com/a/fuli.html'},
 {'title': '长腿美臀', 'url': 'http://www.meizitu.com/a/legs.html'},
 {'title': '日系妹纸', 'url': 'http://www.meizitu.com/a/rixi.html'},
 {'title': '治愈系', 'url': 'http://www.meizitu.com/a/qingchun.html'},
 {'title': '运动girl', 'url': 'http://www.meizitu.com/a/yundong.html'},
 {'title': '清纯', 'url': 'http://www.meizitu.com/a/qingchun.html'},
 {'title': '私房', 'url': 'http://www.meizitu.com/a/sifang.html'},
 {'title': '模特', 'url': 'http://www.meizitu.com/tag/mote_6_1.html'},
 {'title': '可爱', 'url': 'http://www.meizitu.com/tag/keai_64_1.html'},
 {'title': '比基尼', 'url': 'http://www.meizitu.com/a/bijini.html'},
 {'title': '气质', 'url': 'http://www.meizitu.com/tag/qizhi_53_1.html'},
 {'title': '半裸', 'url': 'http://www.meizitu.com/tag/banluo_5_1.html'},


# 获取当前tag的所有html地址

In [19]:
import requests
from bs4 import BeautifulSoup
def get_tag_all_html_link(url):
    tagAllHtml = []
    tagAllHtml.append(url)
    html = requests.get(url)
    html.encoding='gbk'
    bsoup = BeautifulSoup(html.text,'html.parser').find('div',{'class','navigation'}).findAll('a')[:-2]
    if bsoup:
        for a in bsoup:
            if url.startswith('http://www.meizitu.com/a/'):
                tagAllHtml.append('http://www.meizitu.com/a/' + a.attrs['href'])
            else:
                tagAllHtml.append('http://www.meizitu.com' + a.attrs['href'])
    else:
        print('没有下一页！')    
    return tagAllHtml
        
get_tag_all_html_link('http://www.meizitu.com/a/pure.html')

['http://www.meizitu.com/a/pure.html',
 'http://www.meizitu.com/a/pure_2.html',
 'http://www.meizitu.com/a/pure_3.html',
 'http://www.meizitu.com/a/pure_4.html',
 'http://www.meizitu.com/a/pure_5.html',
 'http://www.meizitu.com/a/pure_6.html',
 'http://www.meizitu.com/a/pure_7.html',
 'http://www.meizitu.com/a/pure_8.html',
 'http://www.meizitu.com/a/pure_9.html',
 'http://www.meizitu.com/a/pure_10.html',
 'http://www.meizitu.com/a/pure_11.html',
 'http://www.meizitu.com/a/pure_12.html',
 'http://www.meizitu.com/a/pure_13.html',
 'http://www.meizitu.com/a/pure_14.html',
 'http://www.meizitu.com/a/pure_15.html']

# 模拟header

In [2]:
import random
UserAgent_List = [
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
    "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
    "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
    "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
    "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
    "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"]

def constructHeaders():
    headers = {
    'User-Agent': random.choice(UserAgent_List),
    'Accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    'Accept-Encoding': 'gzip',
    }
    return headers
constructHeaders()

{'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
 'Accept-Encoding': 'gzip',
 'User-Agent': 'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3'}

# 爬取代理ip存入数据库中

In [22]:
import requests
import re

url_proxy = 'http://www.xicidaili.com/nt/'
num = 2 #爬取页数，前两页
def ip_test(ip, headers, url_for_test='https://www.baidu.com', set_timeout=10):
    try:
        rp = requests.get(url_for_test, headers=headers, proxies={'http': ip[0] + ':' + ip[1]}, timeout = set_timeout)
        if rp.status_code == 200:
            return True
        else:
            return False
        
    except BaseException as e:
        return False
    
    
def sraw_ip(url_proxy, num,headers, url_for_test='https://www.baidu.com'):    
    ip_list = []
    dbhelper = DBHelper()
    for num_page in range (1, num +1):
        url_proxy = url_proxy + str(num_page)
        resp = requests.get(url_proxy, headers=headers, timeout = 10)
        resp.encoding = 'utf-8'
        htmlContent = resp.text
        
        pattern = re.compile('<td class="country">.*?alt="Cn" />.*?</td>.*?<td>(.*?)</td>.*?<td>(.*?)</td>',re.S)
        iterms = re.findall(pattern, htmlContent)
        for ip in iterms:
            if ip_test(ip, headers, url_for_test):
                print('测试通过，IP地址为' + str(ip[0]) + ':' + str(ip[1]))
                ip_list.append(ip[0] + ':' + ip[1])
                myDict = {'ip':ip[0] + ':' + ip[1]}
                dbhelper.insert('t_meizitu_proxy_ip', myDict)
            else:
                print('测试失败！')
    dbhelper.close()
    return ip_list

headers = constructHeaders()
sraw_ip(url_proxy, num, headers)
    

测试通过，IP地址为14.29.84.50:8080
测试通过，IP地址为119.90.63.3:3128
测试通过，IP地址为119.129.96.141:9797
测试通过，IP地址为219.245.4.239:3128
测试通过，IP地址为220.180.50.14:53281
测试通过，IP地址为1.192.246.240:9999
测试通过，IP地址为125.46.0.62:53281
测试通过，IP地址为113.89.52.166:9999
测试通过，IP地址为122.72.18.35:80
测试通过，IP地址为202.98.197.243:3128
测试通过，IP地址为113.99.218.73:9797
测试通过，IP地址为27.46.51.171:9797
测试通过，IP地址为183.51.191.127:9999
测试通过，IP地址为1.196.160.20:9999
测试通过，IP地址为122.136.212.132:53281
测试通过，IP地址为61.155.164.108:3128
测试通过，IP地址为58.252.6.165:9000
测试通过，IP地址为113.109.249.22:9797
测试通过，IP地址为222.244.35.120:9000
测试通过，IP地址为1.196.161.20:9999
测试通过，IP地址为113.118.97.119:9797
测试通过，IP地址为116.52.134.5:9999
测试通过，IP地址为221.7.49.209:53281
测试通过，IP地址为220.249.185.178:9999
测试通过，IP地址为123.138.89.133:9999
测试通过，IP地址为61.155.164.109:3128
测试通过，IP地址为112.74.94.142:3128
测试通过，IP地址为121.43.178.58:3128
测试通过，IP地址为122.72.18.34:80
测试通过，IP地址为124.89.33.75:9999
测试通过，IP地址为123.139.56.238:9999
测试通过，IP地址为112.117.59.203:9999
测试通过，IP地址为113.200.214.164:9999
测试通过，IP地址为182.121.204.0:9999
测试通过，IP地址为12

['14.29.84.50:8080',
 '119.90.63.3:3128',
 '119.129.96.141:9797',
 '219.245.4.239:3128',
 '220.180.50.14:53281',
 '1.192.246.240:9999',
 '125.46.0.62:53281',
 '113.89.52.166:9999',
 '122.72.18.35:80',
 '202.98.197.243:3128',
 '113.99.218.73:9797',
 '27.46.51.171:9797',
 '183.51.191.127:9999',
 '1.196.160.20:9999',
 '122.136.212.132:53281',
 '61.155.164.108:3128',
 '58.252.6.165:9000',
 '113.109.249.22:9797',
 '222.244.35.120:9000',
 '1.196.161.20:9999',
 '113.118.97.119:9797',
 '116.52.134.5:9999',
 '221.7.49.209:53281',
 '220.249.185.178:9999',
 '123.138.89.133:9999',
 '61.155.164.109:3128',
 '112.74.94.142:3128',
 '121.43.178.58:3128',
 '122.72.18.34:80',
 '124.89.33.75:9999',
 '123.139.56.238:9999',
 '112.117.59.203:9999',
 '113.200.214.164:9999',
 '182.121.204.0:9999',
 '122.114.122.212:9999',
 '222.66.22.82:8090',
 '202.98.197.242:3128',
 '61.155.164.106:3128',
 '58.243.50.184:53281',
 '120.9.72.14:9999',
 '202.109.207.125:8888',
 '117.121.100.9:3128',
 '123.7.38.31:9999',
 '218.5

# 获取每个tag的所有html地址并保存

In [23]:
def save_all_tags_html_links():
    dbhelper = DBHelper()
    html_links = []
    for tags in getAllTagsFirstLink():
        print(tags['url'])
        allinks = get_tag_all_html_link(tags['url'])
        for urlink in allinks:
            print(tags['title'], urlink)
            myDict={'tag':tags['title'], 'html_url':urlink}
            dbhelper.insert('t_meizitu_tag_html_urls',myDict)
    dbhelper.close()
save_all_tags_html_links()

http://www.meizitu.com/a/pure.html
颜值控 http://www.meizitu.com/a/pure.html
颜值控 http://www.meizitu.com/a/pure_2.html
颜值控 http://www.meizitu.com/a/pure_3.html
颜值控 http://www.meizitu.com/a/pure_4.html
颜值控 http://www.meizitu.com/a/pure_5.html
颜值控 http://www.meizitu.com/a/pure_6.html
颜值控 http://www.meizitu.com/a/pure_7.html
颜值控 http://www.meizitu.com/a/pure_8.html
颜值控 http://www.meizitu.com/a/pure_9.html
颜值控 http://www.meizitu.com/a/pure_10.html
颜值控 http://www.meizitu.com/a/pure_11.html
颜值控 http://www.meizitu.com/a/pure_12.html
颜值控 http://www.meizitu.com/a/pure_13.html
颜值控 http://www.meizitu.com/a/pure_14.html
颜值控 http://www.meizitu.com/a/pure_15.html
http://www.meizitu.com/a/cute.html
萌妹 http://www.meizitu.com/a/cute.html
萌妹 http://www.meizitu.com/a/cute_2.html
萌妹 http://www.meizitu.com/a/cute_3.html
萌妹 http://www.meizitu.com/a/cute_4.html
萌妹 http://www.meizitu.com/a/cute_5.html
萌妹 http://www.meizitu.com/a/cute_6.html
萌妹 http://www.meizitu.com/a/cute_7.html
萌妹 http://www.meizitu.com/a/cute_

美腿 http://www.meizitu.com/a/sifang.html
美腿 http://www.meizitu.com/a/sifang_5_2.html
美腿 http://www.meizitu.com/a/sifang_5_3.html
美腿 http://www.meizitu.com/a/sifang_5_4.html
美腿 http://www.meizitu.com/a/sifang_5_5.html
美腿 http://www.meizitu.com/a/sifang_5_6.html
美腿 http://www.meizitu.com/a/sifang_5_7.html
美腿 http://www.meizitu.com/a/sifang_5_8.html
美腿 http://www.meizitu.com/a/sifang_5_9.html
美腿 http://www.meizitu.com/a/sifang_5_10.html
美腿 http://www.meizitu.com/a/sifang_5_11.html
美腿 http://www.meizitu.com/a/sifang_5_12.html
美腿 http://www.meizitu.com/a/sifang_5_13.html
http://www.meizitu.com/a/qingchun.html
清纯 http://www.meizitu.com/a/qingchun.html
清纯 http://www.meizitu.com/a/qingchun_3_2.html
清纯 http://www.meizitu.com/a/qingchun_3_3.html
清纯 http://www.meizitu.com/a/qingchun_3_4.html
清纯 http://www.meizitu.com/a/qingchun_3_5.html
清纯 http://www.meizitu.com/a/qingchun_3_6.html
清纯 http://www.meizitu.com/a/qingchun_3_7.html
清纯 http://www.meizitu.com/a/qingchun_3_8.html
http://www.meizitu.com/a/

# 获取当前page的相册链接列表

In [15]:
import requests
from bs4 import BeautifulSoup
def get_current_page_albums(page):
    current_page_albums = []
    rep = requests.get(page)
    rep.encoding = 'gb2312'
    bsoup = BeautifulSoup(rep.text, 'html.parser')
    album_list = bsoup.find('ul', {'class', 'wp-list clearfix'}).findAll('li', {'class', 'wp-item'})
    for album in album_list:
        album_link = album.find('h3',{'class', 'tit'}).find('a').attrs['href']
        current_page_albums.append(album_link)
    return current_page_albums
    

# 获取当前相册的图片链接及标题

In [13]:
import requests
import time
from bs4 import BeautifulSoup

def get_current_album_title_imgs(current_page_albums_links, tag_path):
    try:
        for link in current_page_albums_links:
            rep = requests.get(link)
            rep.encoding = 'gb2312'
            bsoup = BeautifulSoup(rep.text, 'html.parser')
            imgs = bsoup.find('div', {'class', 'postContent'}).find('p').findAll('img')
            album_title = bsoup.find('div',{'class', 'metaRight'}).find('a').text
            download_album(tag_path, imgs, album_title)
            time.sleep(5)
    except BaseException as e:
        print('获取每个相册图片 & 标题异常！', e)
        
current_page_albums_links = ['http://www.meizitu.com/a/5529.html', 'http://www.meizitu.com/a/5527.html']
        
get_current_album_title_imgs(current_page_albums_links,'F:\Meizitu\90后')
    

看到这长腿，就算是丁义珍式窗口我也认了，第1张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第2张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第3张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第4张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第5张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第6张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第7张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第8张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第9张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第10张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第11张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第12张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第13张download successfully!
妹子图微信download successfully!
图片下载失败--！ HTTPConnectionPool(host='127.0.0.1', port=60915): Read timed out. (read timeout=30)
内涵图天啊公交车上惊现牛股，第2张download successfully!
内涵图天啊公交车上惊现牛股，第3张download successfully!
内涵图天啊公交车上惊现牛股，第4张download successfully!
内涵图天啊公交车上惊现牛股，第5张download successfully!
内涵图天啊公交车上惊现牛股，第6张download successfully!
内涵图天啊公交车上惊现牛股，第7张download successfully!
内涵图天啊公交车上惊现牛股，第8张download successful

# 创建相应的相册，并下载相册

In [12]:
import os
import time
def download_album(tag_path, imgs, album_title):
    headers = constructHeaders()
    for img in imgs:
        time.sleep(2)
        album_title = format_text(album_title)
        img_src = img.attrs['src']
        title = format_text(img.attrs['alt'])
       
        try:
            if not os.path.exists(os.path.join(tag_path, album_title)):
                os.makedirs(os.path.join(tag_path, album_title))
            os.chdir(tag_path + "\\" + album_title)
            exists = os.path.exists(album_title)
            if not exists:                
                try:
                    img_html = requests.get(img_src, headers=headers, stream=True, timeout=30, verify=True)
                    with open(title + '.jpg', 'wb') as f:                        
                        f.write(img_html.content)
                        f.close()
                        print(title + " download successfully!")
                except BaseException as e:
                    print('图片下载失败--！', e)
        except BaseException as e2:
            print('相册文件构建失败！', str(e2))
            continue
            
            
    

# 文本格式化

In [3]:
#对名字进行处理，如果包含下属字符，则直接剔除该字符
def format_text(text):
    text = text.strip()
    for i in ['\\', '/', ':', '*', '?', '"', '<', '>', '!', '|']:
        while i in text:
            text = text.replace(i, '')
    return text
    

# Main  根据tag创建本地文件夹下载图片

In [ ]:
import os
def  create_tags_dir(base_url):
    dbhelper = DBHelper()
    sql = 'SELECT title from t_meizitu_tags_url ORDER BY title asc'
    result = dbhelper.query(sql)
    try:
        if result:
            #遍历数据库中的每个tag
            for tag in result: 
                if not os.path.exists(os.path.join(base_url, tag[0])):
                    os.makedirs(os.path.join(base_url, tag[0]))
                os.chdir(base_url + '\\' + tag[0])
                sql_tag = 'SELECT html_url FROM t_meizitu_tag_html_urls WHERE tag =' + "'" + tag[0] + "'" + ' ORDER BY html_url asc'
                result_tag = dbhelper.query(sql_tag)
                if not result_tag:
                    print("%s 下没有发现url！", tag)
                else:
                    #处理当前tag的 当前页面 里的相册
                    for page in result_tag:
                        #获得当前页面的 相册链接地址列表
                        current_page_albums_links = get_current_page_albums(page[0])
                        #获取单个相册中图片url 链接地址及 相册名称
                        get_current_album_title_imgs(current_page_albums_links, base_url + '\\' + tag[0])                    
                        
                        
        else:
            print("查询异常！")
    except BaseException as e:
        print('操作异常：',e)
    finally:
        dbhelper.close()    
            
    
create_tags_dir('F:\\Meizitu')

看到这长腿，就算是丁义珍式窗口我也认了，第1张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第2张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第3张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第4张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第5张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第6张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第7张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第8张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第9张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第10张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第11张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第12张download successfully!
看到这长腿，就算是丁义珍式窗口我也认了，第13张download successfully!
妹子图微信download successfully!
内涵图天啊公交车上惊现牛股，第1张download successfully!
内涵图天啊公交车上惊现牛股，第2张download successfully!
内涵图天啊公交车上惊现牛股，第3张download successfully!
内涵图天啊公交车上惊现牛股，第4张download successfully!
内涵图天啊公交车上惊现牛股，第5张download successfully!
内涵图天啊公交车上惊现牛股，第6张download successfully!
内涵图天啊公交车上惊现牛股，第7张download successfully!
内涵图天啊公交车上惊现牛股，第8张download successfully!
内涵图天啊公交车上惊现牛股，第9张download successfully!
内涵图天啊公交车上惊

美女走光，第4张download successfully!
美女走光，第5张download successfully!
美女走光，第6张download successfully!
美女走光，第7张download successfully!
美女走光，第8张download successfully!
美女走光，第9张download successfully!
美女走光，第10张download successfully!
妹子图微信download successfully!
娇羞女生，第1张download successfully!
娇羞女生，第2张download successfully!
娇羞女生，第3张download successfully!
娇羞女生，第4张download successfully!
娇羞女生，第5张download successfully!
娇羞女生，第6张download successfully!
娇羞女生，第7张download successfully!
娇羞女生，第8张download successfully!
娇羞女生，第9张download successfully!
娇羞女生，第10张download successfully!
妹子图微信download successfully!
妹子们，第1张download successfully!
妹子们，第2张download successfully!
妹子们，第3张download successfully!
妹子们，第4张download successfully!
妹子们，第5张download successfully!
妹子们，第6张download successfully!
妹子们，第7张download successfully!
妹子们，第8张download successfully!
妹子们，第9张download successfully!
妹子们，第10张download successfully!
妹子图微信download successfully!
半遮半露，第1张download successfully!
半遮半露，第2张download successfully!
半遮半露，第3张download success

运动美女，第7张download successfully!
妹子图微信download successfully!
健身房里的女孩，第1张download successfully!
健身房里的女孩，第2张download successfully!
健身房里的女孩，第3张download successfully!
健身房里的女孩，第4张download successfully!
健身房里的女孩，第5张download successfully!
健身房里的女孩，第6张download successfully!
妹子图微信download successfully!
妹子美图，第1张download successfully!
妹子美图，第2张download successfully!
妹子美图，第3张download successfully!
妹子美图，第4张download successfully!
妹子美图，第5张download successfully!
妹子美图，第6张download successfully!
妹子美图，第7张download successfully!
妹子美图，第8张download successfully!
妹子美图，第9张download successfully!
妹子美图，第10张download successfully!
妹子图微信download successfully!
模特王曼妮，第1张download successfully!
模特王曼妮，第2张download successfully!
模特王曼妮，第3张download successfully!
模特王曼妮，第4张download successfully!
模特王曼妮，第5张download successfully!
模特王曼妮，第6张download successfully!
模特王曼妮，第7张download successfully!
妹子图微信download successfully!
唯美美女，第1张download successfully!
唯美美女，第2张download successfully!
唯美美女，第3张download successfully!
唯美美女，第4张download successf

妹子图微信download successfully!
上海成人展，第1张download successfully!
上海成人展，第2张download successfully!
上海成人展，第3张download successfully!
图片下载失败--！ HTTPConnectionPool(host='127.0.0.1', port=60915): Read timed out. (read timeout=30)
上海成人展，第5张download successfully!
上海成人展，第6张download successfully!
妹子图微信download successfully!
性感丰满，第1张download successfully!
性感丰满，第2张download successfully!
性感丰满，第3张download successfully!
性感丰满，第4张download successfully!
性感丰满，第5张download successfully!
性感丰满，第6张download successfully!
性感丰满，第7张download successfully!
性感丰满，第8张download successfully!
性感丰满，第9张download successfully!
妹子图微信download successfully!
素颜美女，第1张download successfully!
素颜美女，第2张download successfully!
素颜美女，第3张download successfully!
素颜美女，第4张download successfully!
素颜美女，第5张download successfully!
素颜美女，第6张download successfully!
素颜美女，第7张download successfully!
妹子图微信download successfully!
美女露背，第1张download successfully!
美女露背，第2张download successfully!
美女露背，第3张download successfully!
美女露背，第4张download successfully!
美女露背，第5张downlo

巨乳美女刘飞儿Faye，第8张download successfully!
爆乳妹妹，第1张download successfully!
爆乳妹妹，第2张download successfully!
爆乳妹妹，第3张download successfully!
爆乳妹妹，第4张download successfully!
爆乳妹妹，第5张download successfully!
爆乳妹妹，第6张download successfully!
闺蜜写真，第1张download successfully!
闺蜜写真，第2张download successfully!
闺蜜写真，第3张download successfully!
闺蜜写真，第4张download successfully!
闺蜜写真，第5张download successfully!
闺蜜写真，第6张download successfully!
闺蜜写真，第7张download successfully!
长腿妹妹，第1张download successfully!
长腿妹妹，第2张download successfully!
长腿妹妹，第3张download successfully!
长腿妹妹，第4张download successfully!
长腿妹妹，第5张download successfully!
长腿妹妹，第6张download successfully!
长腿妹妹，第7张download successfully!
长腿妹妹，第8张download successfully!
长腿妹妹，第9张download successfully!
长腿妹妹，第10张download successfully!
大胆模特图片，第1张download successfully!
大胆模特图片，第2张download successfully!
大胆模特图片，第3张download successfully!
大胆模特图片，第4张download successfully!
大胆模特图片，第5张download successfully!
大胆模特图片，第6张download successfully!
长腿正妹街拍，第1张download successfully!
长腿正妹街拍，第2张downloa

很有撸点的图片啊，第9张download successfully!
于大小姐，ayu大美女发福利来咯，第1张download successfully!
新年福利图，第2张download successfully!
福利社图片，第3张download successfully!
2014年美女第一波，第4张download successfully!
让人受不了的全裸，第5张download successfully!
非常棒的身材，第6张download successfully!
用红包遮住重点部位，第7张download successfully!
用红包遮住重点部位，8张download successfully!
用红包遮住重点部位，第9张download successfully!
剪刀手可以很呆萌的哦，第1张download successfully!
素颜美女，第2张download successfully!
小清新mm，第3张download successfully!
素颜美女，第4张download successfully!
素颜美女，第5张download successfully!
可爱清纯，6张download successfully!
粉嘟嘟的小嘴，第7张download successfully!
粉嘟嘟的小嘴，第8张download successfully!
漂亮的妹子，第1张download successfully!
可爱的美女，第2张download successfully!
好白嫩的皮肤，第3张download successfully!
戴红绳是心有所属了吗，第4张download successfully!
窗台的这边是朦胧那换个边呢？，第5张download successfully!
很美很迷人，6张download successfully!
小蛮腰哦，第1张download successfully!
可爱无邪的圣诞女郎，第2张download successfully!
圣诞树下的美女，第3张download successfully!
享受一顿圣诞大餐吧，第4张download successfully!
拿到礼物啦，第5张download successfully!
比基尼女郎，6张downl